# Preprocesar
En este archivo vamos a preprocesar el texto de manera que busquemos la mejor manera de sacar las etiquetas de cada tipo de palabra (POS tagging). El texto que vamos a usar va a estar en español, por lo que tenemos que usar modelos y métodos que lo soporten y tener en cuenta la sintaxis de dicho idioma. 

# 1 Sacar etiquetas
Como hemos mencionado antes, vamos a sacar las etiquetas de cada palabra en un texto, comprobaremos el funcionamiento de varios modelos y discutiremos con cuál nos quedamos.

## 1.1 Imports
El primer paso es definir los imports que vamos a usar durante esta parte, así como descargar datos para dichos modelos.

In [1]:
import spacy
import stanza
from flair.data import Sentence
from flair.models import SequenceTagger

La celda de abajo solo es necesaria una vez.

In [4]:
spacy.cli.download('es_core_news_sm')
print('-'*80)
spacy.cli.download('es_dep_news_trf')
print('-'*80)
stanza.download('es')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------


2026-01-05 19:47:17 INFO: Downloaded file to C:\Users\Jorge\stanza_resources\resources.json
2026-01-05 19:47:17 INFO: Downloading default packages for language: es (Spanish) ...
2026-01-05 19:47:18 INFO: File exists: C:\Users\Jorge\stanza_resources\es\default.zip
2026-01-05 19:47:22 INFO: Finished downloading models and saved to C:\Users\Jorge\stanza_resources


## 1.2 Corpus de prueba
Antes de pasar a la acción definimos un corpus de prueba para evaluar los modelos.En este vamos a tener todas las frases que vayamos a usar a lo largo de todo el preprocesamiento


In [9]:
corpus = [
    'El gato come pescado.',
    'El equipo de fútbol jugó su partido. Este ganó con facilidad.',
    'El coche rojo se estropeó, así que lo llevé al taller.',
    'La presidenta y el director se reunieron; ella habló primero.',
    'Entregué el informe a la jefa después de que ella lo leyera.',
    'Los equipos trabajaron duro, y al final ellos ganaron el premio.',
    'A pesar de sus problemas, el artista terminó su obra.',
    'El hermano de María dijo que él vendría.',
    'En su oficina, el abogado revisó los documentos.',
    'El libro que leí es fascinante; este autor siempre sorprende.',
    'El gato persiguió al ratón, pero este logró escapar.',
    'Hablé con Pedro sobre su proyecto y luego él me envió los archivos.',
    'Ayer hablé con Juan, le dije: "dímelo, por favor", y no me lo quiso decir',
    'Dímelo',
    'Dí me lo',
    'Di me lo',
    'Él vino a dármelo',
    "La cabra de mi amigo Juan arremete todos los días contra la valla.",
    "Mi madre me dijo: 'cómete la tarta', por lo que me la comí.",
    "Mi profesor me dijo: 'comete este error y suspenderás', por lo que tuve mucho cuidado."
    "No sé si viste ayer a mi padre, pero yo sí.",
    "Vístete rápido, que ya nos vamos.",
    "Muéstrame tu chaqueta nueva."
]

## 1.3 Pruebas
Vamos a empezar a probar modelos sobre todas las frases que hemos definido antes.

### 1.3.1 Spacy - es_core_news_sm

In [10]:
nlp = spacy.load("es_core_news_sm")
test1 = []
for text in corpus: test1.append(nlp(text))
for doc in test1:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado ADJ
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué PROPN
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En general lo hace muy bien, aunque confunde VERB que comienzan una oración con PROPN (sustantivos propios) y, en la primera oración, confunde el sustantivo pescado por el adjetivo, por lo que podemos intuir que cometerá más veces ese error. Por lo que a nuestra futura tarea respecta, el primer error puede ser garrafal, pues podría pensar el modelo que un pronombre se refiere a un verbo de esos mal etiquetados. Además, el verbo decir más los enclíticos me y lo lo considera un sustantivo, fallo enorme ya que son dos pronombres tras el verbo.
Veamos el resto de modelos.

### 1.3.2 Spacy - es_dep_news_trf

In [11]:
nlp = spacy.load("es_dep_news_trf")
test2 = []
for text in corpus: test2.append(nlp(text))
for doc in test2:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En este caso vemos que corrige todos los errores previos, salvo el caso de los enclíticos, que los considera verbos (bien, pero solo la raíz es el verbo). Esto muestra que este modelo es mejor, pero no perfecto, pues confunde, por ejemplo "Dí", de separar los enclíticos en "Dímelo", por AUX, pero sin la tilde dice correctamente que es un verbo, esto lo tendremos en cuenta más adelante.

### 1.3.3 Spacy - es_dep_news_trf (texto en minúsculas)

In [12]:
nlp = spacy.load("es_dep_news_trf")
test3 = []
for text in corpus: test3.append(nlp(text.lower()))
for doc in test3:
    for token in doc:
        print(token.text, token.pos_)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DET
libro NOU

Al hacer varias pruebas nos damos cuenta que estos modelos son <em>case sensitive</em>, es decir, las palabras en mayúsculas y minúsculas importan. En este caso vemos que se solventa el problema del AUX en "dí", pero aún así palabras como "dímelo" no las hace bien. Probemos con otro modelo, en este caso Stanza.

### 1.3.4 Stanza

In [ ]:
nlp = stanza.Pipeline('es', verbose=False)
test4 = []
for text in corpus: test4.append(nlp(text))
for doc in test4:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El D

En este caso el modelo funciona muy similar a Spacy con es_dep_news_trf. Tampoco identifica correctamente los pronombres en "dímelo", y en este caso lo marca como interjección (INTJ). Cuando solo ponemos el verbo con enclíticos (Dímelo) lo considera verbo, y cuando lo separamos con espacios las partes analiza bien los pronombres pero el verbo vuelve a confundirlo con interjección, aunque si el verbo está sin tilde lo sigue marcanto como INTJ. Probemos con el texto en minúsculas. 

### 1.3.5 Stanza (minúsculas)

In [ ]:
nlp = stanza.Pipeline('es', verbose=False)
test5 = []
for text in corpus: test5.append(nlp(text.lower()))
for doc in test5:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOUN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DE

Aquí cambia ligeramente: "di" lo marca bien como verbo, mientras que "dímelo" lo marca como sustantivo, y lo demás se mantiene igual. Aunque sea parecido a Spacy, este parece menos consistente con verbos con enclíticos. Ahora probamos con el último modelo, Flair.

### 1.3.6 Stanza (distinto Pipeline)

In [ ]:
nlp = stanza.Pipeline(
    "es",
    processors="tokenize,mwt,pos,lemma",
    tokenize_no_ssplit=False, 
)

for text in corpus:
    doc = nlp(text)
    for sent in doc.sentences:
        for w in sent.words:
            print(w.text, w.lemma, w.upos)#, w.feats)
        print()

In [27]:
# 2. Inicializar Pipeline
nlp = stanza.Pipeline(
    lang='es',
    processors='tokenize,mwt,pos,lemma,ner',
    use_gpu=False,
    pos_batch_size=2000
)

# 3. Procesar texto
texto = "Dármelo ahora, pásalo bien por favor."
texto2 = "Me gusta mi coche, ayer vine a comprarlo."
texto3 = "Me pasé toda la tarde jugándolo"
doc = nlp(texto2)

# 4. Imprimir resultados con POS y lemma
for sent in doc.sentences:
    for word in sent.words:
        print(word.text, word.lemma, word.upos, word.xpos, word.ner)

AttributeError: 'Word' object has no attribute 'ner'

### 1.3.7 Flair

In [20]:
# Cargar modelo POS (multilingual)
tagger = SequenceTagger.load("pos-multi")

for text in corpus:
    sentence = Sentence(text.lower())
    tagger.predict(sentence)
    for token in sentence:
        print(token.text, token.get_labels()[0].value)
    print()

2026-01-05 20:03:17,324 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que CCONJ
lo PRON
llevé VERB
al DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADJ
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOU

En este modelo vemos más inconsistencias todavía. "di" lo marca como ADP, y "lo" al separar "dímelo" lo marca como verbo. No mejora en ningún aspecto a los anteriores. <br>
Para mejorar este comportamiento vamos a hacer un preprocesamiento personalizado, el cual tratará de separar los enclíticos, pero para ello tenemos que ver qué verbos tienen enclíticos y cuáles no.

# 2 Preprocesamiento personalizado
Tras varias pruebas pensamos que el Spacy con dep news (texto en minúsculas) es el más consistente, ya que solo falla en los enclíticos. Para mejorar con los enclíticos podemos hacer nuestro propio preprocesamiento al texto para que así lo etiquete como debe.  
En primer lugar vamos a probar a utilizar un stemmer para sacar la raíz de las palabras, de esta forma vamos a ver si las etiquetas se mantienen como antes y, además, resuelve los verbos con enclíticos y los verbos con falsos enclíticos (i.e. verbos como "comete", "viste", etc.). Idealmente queremos que se lematicen los verbos sin enclíticos y los que sí los tienen los deje igual para luego separarlos. 
Si esto no funciona como esperamos entonces cogeremos la lista de pronombres que pueden ir junto a los verbos y los ponemos en orden (i.e. nunca se tiene "lasme", siempre "melas", como en "dámelas"). Pero esto solo no es suficiente, puesto que hay verbos que tienen "me", "te" u otros. Pero comete es diferente de cómete, de comer, por lo que necesitamos una lista con verbos a los que no les tenemos que aplicar este preprocesamiento. 

## 2.1 Imports
Primero definimos los imports que usaremos en esta parte, al igual que hicimos antes.

In [35]:
from nltk.stem import SnowballStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize

## 2.2 Funciones auxiliares
Para no repetir código, vamos a definir una o varias funciones auxiliares.

In [36]:
def pos_tag(corpus: list[str] = []):
    nlp = spacy.load("es_dep_news_trf")
    docs = []
    for text in corpus: docs.append(nlp(text.lower()))
    return docs 

In [37]:
def compare_tags(docs: list, stems: list, compare: list):
    for doc, cmp, stem_phrase in zip(docs, compare, stems):
        for doc_token, cmp_token, stem in zip(doc, cmp, stem_phrase):
            if doc_token.pos_ != cmp_token.pos_:
                print('-'*40, '\n', 
                      doc_token.text, stem, doc_token.pos_, '|', cmp_token.text, cmp_token.pos_,
                      '\n', '-'*40)
            print(doc_token.text, stem, doc_token.pos_, '|', cmp_token.text, cmp_token.pos_)
        print() 

In [38]:
def print_tags(docs: list = [], stems: list = [], compare: list | None = None):
    if compare:
        compare_tags(docs, stems, compare)
    else:
        for doc in docs:
            for token, stem in zip(doc, stems):
                print(token.text, stem, token.pos_)
            print()    

## 2.3 Pruebas
Vamos ahora a probar los modelos para arreglar el fallo con los enclíticos.

### 2.3.1 LancasterStemmer

In [39]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = LancasterStemmer()
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])

docs = pos_tag(corpus)
print_tags(docs, stems, test3)

el el DET | el DET
gato gato NOUN | gato NOUN
come com VERB | come VERB
pescado pescado NOUN | pescado NOUN
. . PUNCT | . PUNCT

el el DET | el DET
equipo equipo NOUN | equipo NOUN
de de ADP | de ADP
fútbol fútbol NOUN | fútbol NOUN
jugó jugó VERB | jugó VERB
su su DET | su DET
partido partido NOUN | partido NOUN
. . PUNCT | . PUNCT
este est PRON | este PRON
ganó ganó VERB | ganó VERB
con con ADP | con ADP
facilidad facilidad NOUN | facilidad NOUN
. . PUNCT | . PUNCT

el el DET | el DET
coche coch NOUN | coche NOUN
rojo rojo ADJ | rojo ADJ
se se PRON | se PRON
estropeó estropeó VERB | estropeó VERB
, , PUNCT | , PUNCT
así así ADV | así ADV
que que SCONJ | que SCONJ
lo lo PRON | lo PRON
llevé llevé VERB | llevé VERB
al al ADP | al ADP
taller tal NOUN | taller NOUN
. . PUNCT | . PUNCT

la la DET | la DET
presidenta president NOUN | presidenta NOUN
y y CCONJ | y CCONJ
el el DET | el DET
director direct NOUN | director NOUN
se se PRON | se PRON
reunieron reunieron VERB | reunieron VERB
; ;

Vemos que evalúa exactamente igual que con el texto. Además, algunos verbos con enclíticos los lematiza de forma que se pierde el pronombre (e.g. muéstrame lo lematiza como muéstram), por lo que no nos sirve este modelo.

### 2.3.2 SnowballStemmer

In [50]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = SnowballStemmer("spanish")
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])

docs = pos_tag(corpus)
print_tags(docs, stems, test3)

el el DET | el DET
gato gat NOUN | gato NOUN
come com VERB | come VERB
pescado pesc NOUN | pescado NOUN
. . PUNCT | . PUNCT

el el DET | el DET
equipo equip NOUN | equipo NOUN
de de ADP | de ADP
fútbol futbol NOUN | fútbol NOUN
jugó jug VERB | jugó VERB
su su DET | su DET
partido part NOUN | partido NOUN
. . PUNCT | . PUNCT
este este PRON | este PRON
ganó gan VERB | ganó VERB
con con ADP | con ADP
facilidad facil NOUN | facilidad NOUN
. . PUNCT | . PUNCT

el el DET | el DET
coche coch NOUN | coche NOUN
rojo roj ADJ | rojo ADJ
se se PRON | se PRON
estropeó estrope VERB | estropeó VERB
, , PUNCT | , PUNCT
así asi ADV | así ADV
que que SCONJ | que SCONJ
lo lo PRON | lo PRON
llevé llev VERB | llevé VERB
al al ADP | al ADP
taller tall NOUN | taller NOUN
. . PUNCT | . PUNCT

la la DET | la DET
presidenta president NOUN | presidenta NOUN
y y CCONJ | y CCONJ
el el DET | el DET
director director NOUN | director NOUN
se se PRON | se PRON
reunieron reun VERB | reunieron VERB
; ; PUNCT | ; PUNCT
e

Este caso funciona igual que el anterior.

In [ ]:
from itertools import product

CLITICS = ['me','te','se','lo','la','los','las','le','les','nos','os']

# construye combos posibles (hasta 2 clíticos en la práctica; hay casos de 3 pero raros)
def generate_clitic_combos():
    combos = set()
    # uno
    for c in CLITICS:
        combos.add(c)
    # dos (orden correcto: primero clítico átono como me/te/se/nos/ se + lo/la/los/las/le/les)
    for a in CLITICS:
        for b in CLITICS:
            combos.add(a + b)
    # opcional: podrías añadir triples si lo necesitas
    return sorted(combos, key=lambda x: -len(x))  # orden largo -> corto

CLITIC_COMBOS = generate_clitic_combos()

def analyze_text(nlp, text):
    """Analiza text y devuelve el primer word analysis (upos, feats, lemma) si existe."""
    doc = nlp(text)
    # si hay oraciones y words:
    if doc.sentences and doc.sentences[0].words:
        w = doc.sentences[0].words[0]
        return {'text': w.text, 'upos': w.upos, 'feats': w.feats or '', 'lemma': w.lemma}
    return None

def is_pronoun_like(nlp, token_text):
    a = analyze_text(nlp, token_text)
    return a is not None and (a['upos'] == 'PRON' or a['upos'] == 'DET')  # DET para artículos si aplicara

def try_split_token(nlp, token_text):
    """
    Devuelve lista de sub-tokens si decide dividir, 
    o [token_text] si no dividir.
    """
    token_text_orig = token_text
    token_text_lower = token_text_orig.lower()

    # 1) analiza token completo
    full = analyze_text(nlp, token_text_orig)
    # Si Stanza lo considera verbo 3ª pers. indicativo -> probablemente no es enclítico
    if full and full['upos'] == 'VERB' and 'Person=3' in full['feats'] and 'Mood=Imp' not in (full['feats'] or ''):
        return [token_text_orig]  # no dividimos

    # 2) intentamos splits por combinaciones de clíticos (de más largo a más corto)
    for combo in CLITIC_COMBOS:
        if token_text_lower.endswith(combo):
            left = token_text_orig[:len(token_text_orig)-len(combo)]
            right = token_text_orig[len(token_text_orig)-len(combo):]
            if not left: 
                continue

            # valida que cada pieza derecha se componga en pronombres conocidos (por si combo es raro)
            # descomponer right en secuencias de clíticos (intento simple: probar particion en 1..n)
            # para simplicidad, si combo está en la lista CLITICS o es concatenación válida, lo aceptamos provisionalmente
            # comprobación morfológica:
            left_a = analyze_text(nlp, left)
            right_a = analyze_text(nlp, right)

            # Conditions to accept split:
            # - left exists and is VERB
            # - right exists and is PRON (o sequence of PRONs)  (aquí cheque simple)
            if left_a and left_a['upos'] == 'VERB' and right_a and right_a['upos'] == 'PRON':
                # Heurística adicional: el verbo izquierdo debe mostrarse como imperativo o 2ª persona
                feats = (left_a['feats'] or '')
                if 'Mood=Imp' in feats or 'Person=2' in feats or 'VerbForm=Fin' in feats:
                    return [left, right]
                # otra heurística: si la forma original NO era verbo 3ª persona (evitamos 'comete' -> 'come+te')
                if not (full and full['upos']=='VERB' and 'Person=3' in full['feats']):
                    return [left, right]
                # otherwise continue searching combos

    # si no encontramos nada aceptable
    return [token_text_orig]


# Ejemplo de uso
texts = ["Él comete errores.", "Vete al norte.", "Cómete la manzana.", "comete", "dinos"]
for text in corpus:
    print("===", text)
    # tokeniza rápido por espacio (ejemplo); en uso real recorre doc.sentences.tokens
    for tok in text.split():
        subtoks = try_split_token(nlp, tok)
        print(tok, "->", subtoks)
    print()


In [ ]:
# Lista de clíticos españoles
CLITICOS = ["me","te","se","lo","la","los","las","le","les","nos","os"]

# Orden típico permitido en español
# (simplificado pero suficientemente bueno para casi todos los casos)
ORDEN_CLITICOS = [
    ["me","lo"], ["me","la"], ["me","los"], ["me","las"],
    ["te","lo"], ["te","la"], ["te","los"], ["te","las"],
    ["se","lo"], ["se","la"], ["se","los"], ["se","las"],
    ["nos","lo"], ["nos","la"], ["nos","los"], ["nos","las"],
    ["os","lo"], ["os","la"], ["os","los"], ["os","las"],
]

# Generar patrones de clíticos pegados (ej: "melo", "telo", "selo")
PATRONES = sorted(
    [ "".join(p) for p in ORDEN_CLITICOS ] + CLITICOS,
    key=len, reverse=True
)

VERBOS = [
    "comete",
    "somete",
    "arremete",
    "promete",
    "remete",
    "entromete",
    "admite",
    "emite",
    "omite",
    "remite",
    "permite",
    "transmite",
    "dimite",
    "limite",
    "intermite",
    "retransmite",
    "siente",
    "consiente",
    "presiente",
    "resiente",
    "asiente",
    "disiente",
    "resume",
    "asume",
    "presume",
    "consume",
    "subsume",
    "come",
    "vale",
    'comeríamos'
] # Aún faltan muchísimos, igual mejor guardarlo como txt

In [ ]:
def separar_cliticos(token, lemma):
    """
    Separa clíticos de una forma verbal como 'dímelo', 'háblalo', 'cómetelo', etc.
    """
    t = token.lower()

    # Buscar si termina en un clítico válido (simple o doble)
    encontrados = []
    resto = t

    for patron in PATRONES:
        if resto.endswith(patron):
            encontrados.append(patron)          # ej: "melo"
            resto = resto[: -len(patron)]
            break

    # Si no encontró nada → no hay clíticos pegados
    if not encontrados:
        return [token]  

    # Convertir el verbo a su forma verbal (usar lemma como base)
    raiz = lemma

    # Normalizar la raíz para modo imperativo (caso típico)
    # Esto no es perfecto, pero funciona para mayoría:
    if raiz.endswith("r"):      # comer → come
        raiz = raiz[:-1]
    elif raiz.endswith("irse"): # "irse" → "ir" pero con reflexivos es complejo
        raiz = raiz[:-3]

    resultado = [raiz]

    # Ahora separar los clíticos (simple o doble)
    cl = encontrados[0]

    # Si es doble clítico ("melo", "selo", etc.)
    for c in CLITICOS:
        if cl.startswith(c):
            resto2 = cl[len(c):]
            if c in CLITICOS:
                resultado.append(c)
            if resto2 in CLITICOS:
                resultado.append(resto2)
            break
    
    return resultado

### OTRA FORMA DE SEPARAR CLÍTICOS - LISTA DE EXCEPCIONES + REGLAS

Como podemos observar, stanza tiene bastantes carencias a la hora de reconocer y separar los clíticos en español siendo que solo es bueno reconociendo los infinitivos con clíticos y los gerundios cuando solo tienen un clítico, si presentan doble clítico no los reconoce. Por eso, cualquier forma verbal en imperativo con clíticos fallará a la hora de reconocerla.

#### EJEMPLO VERBO JUGAR

| Forma verbal | Descripción | Stanza (salida incorrecta) | Deseado (análisis correcto) |
|-------------|-------------|----------------------------|-----------------------------|
| juégalo | Imperativo 2ª persona singular + 1 clítico | juégalo → juégalo (NOUN) | juega (jugar, VERB) + lo (él, PRON) |
| juégatelo | Imperativo 2ª persona singular + 2 clíticos | juégatelo → juégatelo (NOUN) | juega (jugar, VERB) + te (tú, PRON) + lo (él, PRON) |
| jugadle | Imperativo 2ª persona plural + 1 clítico | jugadle → jugadle (NOUN) | jugad (jugar, VERB) + le (él, PRON) |
| jugádnoslo | Imperativo 2ª persona plural + 2 clíticos | jugadno → jugadno (NOUN) + lo (él, PRON) | jugad (jugar, VERB) + nos (nosotros, PRON) + lo (él, PRON) |
| juéguelo | Imperativo 2ª persona singular (usted) + 1 clítico | juéguelo → juéguelo (NOUN) | juegue (jugar, VERB) + lo (él, PRON) |
| juégueselo | Imperativo 2ª persona singular (usted) + 2 clíticos | juégueselo → juégueselo (NOUN) | juegue (jugar, VERB) + se (él, PRON) + lo (él, PRON) |
| jueguenlo | Imperativo 2ª persona plural (ustedes) + 1 clítico | jueguenlo → jueguenlo (NOUN) | jueguen (jugar, VERB) + lo (él, PRON) |
| jueguenselo | Imperativo 2ª persona plural (ustedes) + 2 clíticos | jueguenselo → jueguenselo (NOUN) | jueguen (jugar, VERB) + se (él, PRON) + lo (él, PRON) |
| juguémoslo | Imperativo 1ª persona plural + 1 clítico | juguémoslo → juguémoslo (NOUN) | juguemos (jugar, VERB) + lo (él, PRON) |
|juguémonoslo | Imperativo 1ª persona plural + 2 clítico | juguémonoslo -> juguémonoslo NOUN |juguemos (jugar, VERB) + nos (nosotros, PRON) + lo (él, PRON)|
jugándomelo | Gerundio + 2 clíticos | jugando -> jugar VERB + melo -> melo NOUN | jugando (jugar, VERB) + me (yo, PRON) + lo (él, PRON)|

Para ello meteremos estas formas verbales en una lista de excepciones para separarlas de forma manual y así pueden ser tokenizadas correctamente

#### IMPORTS
- re: Librería para expresiones regulares
- stanza: Librería para tokenizar texto en español
- List y Dict: Librerías que representan listas y diccionarios

In [ ]:
import re
import stanza
from typing import List, Dict

#### LISTA DE VERBOS CON CLÍTICOS

Lista de excepciones que el tokenizador de stanza no sabe tokenizar, típicamente son las formas imperativas de los verbos en español que presentan clíticos.

Los verbos en la lista se reconocen mediante expresiones regulares de la base de la palabra y todas las combinaciones de clíticos que nos podemos encontrar en ella. También se especifica de que verbo viene ya que la base puede coincidir con alguna forma verbal de algún otro verbo como sucede con "díselo" cuya base es di que coincide con el pasado del verbo "dar". Por eso se le indica el lema al que pertenece y el upos ya que aunque todos sean verbos, el tokenizador no lo reconoce e intenta adivinarlo dándole alguna upos aleatoria.

In [ ]:
IMPERATIVE_MAP = {
    # Verbo comer
    r"(?:C|c)óme((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "come",
        "lemma": "comer",
        "upos": "VERB"
    },
    r"(?:C|c)om[eé]d((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "comed",
        "lemma": "comer",
        "upos": "VERB"
    },
    # Verbo cometer
    r"(?:C|c)ométe((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "comete",
        "lemma": "cometer",
        "upos": "VERB"
    },
    r"(?:C|c)omet[eé]d((?:me|te|se|nos|os)?)((?:lo|le|la|los|les|las)?)": {
        "base": "cometed",
        "lemma": "cometer",
        "upos": "VERB"
    },
    # Verbo vestir
    r"(?:v|V)íste((?:me|te|se|nos|os)?)((?:lo|la|los|las)?)": {
        "base": "viste",
        "lemma": "vestir",
        "upos": "VERB"
    },
    # Verbo acostar
    r"(?:a|A)cués(?:ta|te)((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "acuesta",
        "lemma": "acostar",
        "upos": "VERB"
    },
    r"(?:a|A)costad((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "acosta",
        "lemma": "acostar",
        "upos": "VERB"
    },
    # Verbo decir
    r"(?:d|D)[íi]((?:me|te|se|nos|os)?)((?:le|lo|la|les|los|las)?)": {
        "base": "di",
        "lemma": "decir",
        "upos": "VERB"
    },
    # Verbo mostrar
    r"(?:m|M)uéstra((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|le)?)": {
        "base": "muestra",
        "lemma": "mostrar",
        "upos": "VERB"
    },
    # Verbo jugar: Contempla todas las formas verbales en la que falla stanza
    r"(?:j|J)ugu[ée]mo((?:nos))((?:lo|la|le|los|las|les)?)": {
        "base": "juguemo",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)uéguen((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "jueguen",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)uguémos((?:lo|la|le|los|las|les)?)": {
        "base": "juguemos",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)uég(?:a|ue)((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "juega",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)ugáz((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "jugad",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)ugáos((?:lo|la|los|las))": {
        "base": "jugad",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)ugándo((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "jugando",
        "lemma": "jugar",
        "upos": "VERB"
    },
    # Añadir más verbos según necesidad
}

Lista de clíticos del español

In [ ]:
CLITICS = ["me", "te", "se", "nos", "os", "lo", "la", "le", "los", "las", "les"]

**split_clitics**

Separa los clíticos de la palabra original

In [ ]:
def split_clitics(clitic_string: str):
    clitics = []
    remaining = clitic_string

    for cl in CLITICS:
        if remaining.startswith(cl):
            clitics.append(cl)
            remaining = remaining[len(cl):]

    return clitics


**preprocess_imperatives**

Coordina el proceso de limpieza del texto. Se descompone en 3 fases:

 1º- Inicialización donde crea una lista (imperative_meta) que guardará la información de cada verbo que se modifique

 2º- El Ciclo de Transformación donde recorrerá el IMPERATIVE_MAP con los patrones y las reglas de sustitución

 3º- Actualización donde aplicará los cambios al texto original devolviéndolo junto con sus metadatos recolectados

**make_replacer**

Se corresponde con el ciclo de transformación y se encargará de coger cada patrón junto con su regla asociada del IMPERATIVE_MAP. Además también devuelve la expresión regular lista para emplearse y la función replacer configurada.

**replacer**

Función que no llama el usuario sino que la llama el motor de expresiones regulares de python cada vez que encuentra una coincidencia que encaja con el patrón. Su funcionamiento es el siguiente:

 1º- Captura los clíticos de la palabra, si no los hay, devuelve la palabra sin modificar

 2º- Crea una nueva cadena de texto separando la raíz del verbo de los pronombre empleando espacios

 3º- Guarda en imperative_meta la información técnica del verbo para que no se pierda la información que había aunque se cambie la palabra

 4º- Asegura que respeta las mayúsculas que la palabra original tenía

 5º- Devuelve la cadena separada para que se inserte en el texto originial

In [ ]:
def preprocess_imperatives(text: str):
    imperative_meta = []

    def make_replacer(pattern: str, rule: Dict):
        regex = re.compile(pattern, flags=re.IGNORECASE) # Carga la expresión regular del patrón

        def replacer(match): # Si no coincide con el patrón, devuelve la palabra sin modificar
            groups = match.groups() # Analiza el patrón para buscar que clíticos tiene
            # Mete los clíticos encontrados como elementos de una lista
            clitic_string = "".join(g for g in groups if g) if groups else ""
            clitics = split_clitics(clitic_string)

            # Si no hay clíticos, no tocar la palabra original
            if not clitics:
                return match.group(0)
            
            # Aplicar la regla correspondiente si hay clíticos
            replacement = " ".join([rule["base"]] + clitics)

            imperative_meta.append({
                "base": rule["base"],
                "lemma": rule["lemma"],
                "upos": rule["upos"]
            })

            # Mantener mayúscula inicial
            if match.group(0)[0].isupper():
                replacement = replacement.capitalize()

            return replacement

        return regex, replacer

    for pattern, rule in IMPERATIVE_MAP.items():
        regex, replacer = make_replacer(pattern, rule)
        text = regex.sub(replacer, text) # Aplica la sustitución al texto original

    return text, imperative_meta

**parse_text**

Función que separá los clíticos llamando a preprocess_imperatives para despues pasarlos por el tokenizador de stanza.pipeline y así tener ya cada palabra tokenizada para luego pasarla al formato final

In [ ]:
def parse_text(text: str):
    # Separar clíticos
    preprocessed, imperative_meta = preprocess_imperatives(text)

    # Tokenizar las palabras
    doc = nlp(preprocessed)
    
    return doc, imperative_meta

**texto_a_frases**

Convierte texto bruto en una lista de frases. Separa el texto cuando encuentra algún signo de puntuación que indique el fin de frase.

Esto es necesario para llamar a la función texts_to_conllu (Notebook de formato conllu) ya que espera una lista

In [ ]:
def texto_a_frases(texto: str) -> list[str]:
    if not texto or not texto.strip():
        return []

    # Normalizar espacios
    texto = re.sub(r'\s+', ' ', texto.strip())

    # Separar por fin de frase
    frases = re.split(r'(?<=[.!?¿¡])\s+', texto)

    # Limpiar frases vacías
    return [f.strip() for f in frases if f.strip()]

# 3 Dataset
Ahora vamos a buscar un dataset para sacar sus etiquetas part-of-speech (POS TAGs). Este dataset será el que después probaremos qué tal sirve nuestro modelo para resolver las referencias de los pronombres a los sustantivos. Hemos encontrado en Hugging Face un dataset llamado CulturaX, que tiene datos en más de 160 idiomas, entre ellos el español. Consideramos que es un dataset lo suficientemente completo con multitud de textos de diferentes procedencias para usarlo en nuestro proyecto. Vamos a coger un subset de varios miles de textos ya que el completo tiene millones o más, y no necesitamos tantos.

## 3.1 Imports
Vamos a ver los imports que necesitamos y el login de Hugging Face.
Si se quiere ejecutar la celda de abajo hay que seguir las instrucciones del login en notebook, es bastante intuitivo.

In [13]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset

## 3.2 Cargar textos
Ahora vamos a cargar los textos en una variable 'ds_es', la cual funciona como Stream para no descargar los datos, solo metadatos que permiten su uso, y después vamos a tomar trivialmente 5 mil textos para nuestro proyecto.

In [14]:
ds_es = load_dataset(
    "uonlp/CulturaX",  # Nombre del dataset
    "es",              # Config de idioma español
    streaming=True     # Stream para no descargar completo
)

Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

In [15]:
# Tomar solo los primeros n ejemplos
n = 5000  
subset = ds_es["train"].take(n)

# Convertir a lista si quieres verlos o trabajar con ellos
data = list(subset)

print(len(data))  # debería dar n

5000


## 3.3 Mostrar datos
Para ver un poco los datos vamos a mostrar unos cuantos textos, primero veamos que tipo es cada cosa y cómo podemos acceder a dichos textos.

In [17]:
print(type(data[1]))

<class 'dict'>


In [18]:
print(data[1].keys())

dict_keys(['text', 'timestamp', 'url', 'source'])


Vemos que tiene 4 claves. Veamos cada campo, excepto texto.

In [21]:
print(data[1]['url'])
print(data[1]['source'])
print(data[1]['timestamp'])

https://periodicoalminutosv.com/5072/nacionales/presidente-bukele-es-bien-evaluado-en-los-primeros-meses-de-su-gestion-90-de-la-poblacion-opina-que-esta-ayudando/
OSCAR-2109
2021-03-04T07:09:46Z


Se entiende bastante bien qué es cada cosa, excepto la fuente. Vamos a mostrar ahora los primeros 5 textos.

In [28]:
for i in range(5):
    print(data[i]['text'], '\n')
    

Gobierno confía en que Corte Constitucional dé vía libre al mecanismo fast track - Eje21
Gobierno confía en que Corte Constitucional dé vía libre al mecanismo fast track
Bogotá, 04 de diciembre _ RAM_ Así lo confirmó este sábado el ministro del Interior Juan Fernando Cristo, quien señaló que: "el Gobierno emprenderá el camino de implementación del nuevo acuerdo de paz que decida la Corte Constitucional", y confía en que la decisión sobre la vía rápida para la implementación de las leyes de paz se tome antes de la vacancia judicial y el fin de esta legislatura en el Congreso, "tenemos la fe de que va a dar salidas que garanticen implementación rápida de los acuerdos". 

hola yo estoy pasando por la misma situacion el 19 de diciembre pedi en mandarake ub shin mazinger , armadura blindada para vf 25 1/60 y el set para el vf 25 tornado y hasta el dia de hoy la pgina de correos de mexico solo dice recibido en oficina postal 2 evntos, la verdad no se para cuando llegue mi paquete es la prime

Podemos ver que hay algunos textos con erratas, y con lenguas de hispanoamérica, lo cual puede ser interesante para ver el desempeño del etiquetado.

## 3.3 Almacenar datos
Ahora vamos a guardar los datos en un archivo y en el siguiente notebook sacaremos las etiquetas.

In [ ]:
filename = "CulturaX"
with open(filename, "w+", encoding="utf-8") as f:
    f.write(data)
print(f"Archivo guardado correctamente como {filename}")